In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torchvision import datasets, transforms
import torch.nn.functional as F
import torch.optim as optim

OSError: [WinError 126] 找不到指定的模块。 Error loading "D:\Programming\python\Anaconda\envs\AI\lib\site-packages\torch\lib\torch_python.dll" or one of its dependencies.

In [ ]:
# Hyper param
epoch_num = 20
batch_size = 64
lr = 1e-3
class_num = 7

In [ ]:
# Read data
train_dir = '../input/emotion-detection-fer/train'
test_dir = '../input/emotion-detection-fer/test'

trainTransform  = transforms.Compose([transforms.Grayscale(num_output_channels=1), transforms.ToTensor()])

train_dataset = datasets.ImageFolder(train_dir,trainTransform)
test_dataset = datasets.ImageFolder(test_dir,trainTransform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
next(iter(train_loader))

In [ ]:
# Model
class CNN(nn.Module):
    def __init__(self,):
        super(CNN, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 128, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer4 = nn.Sequential(
            nn.Conv2d(512, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU()
        )

        self.linear = nn.Linear(12 * 12 * 16, class_num)

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = out.reshape(out.size(0), -1)
        return self.linear(out)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:

train_accs = []
train_loss = []
test_accs = []
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = net.to(device)
for epoch in range(20):
    running_loss = 0.0
    for i,data in enumerate(dataloader,0):#0是下标起始位置默认为0
        # data 的格式[[inputs, labels]]
#         inputs,labels = data
        inputs,labels = data[0].to(device), data[1].to(device)
        #初始为0，清除上个batch的梯度信息
        optimizer.zero_grad()

        #前向+后向+优化
        outputs = net(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        # loss 的输出，每个一百个batch输出，平均的loss
        running_loss += loss.item()


        # 训练曲线的绘制 一个batch中的准确率
        correct = 0
        total = 0
        _, predicted = torch.max(outputs.data, 1)
        total = labels.size(0)# labels 的长度
        correct = (predicted == labels).sum().item() # 预测正确的数目
        train_accs.append(100*correct/total)
        if i%100 == 99:
            print('[%d,%5d] loss :%.3f' %
                 (epoch+1,i+1,running_loss/100))
            print(predicted, labels)
            running_loss = 0.0
        train_loss.append(loss.item())

for i, data in enumerate(testloader, 0):
    inputs,labels = data[0].to(device), data[1].to(device)
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0)# labels 的长度
    correct = (predicted == labels).sum().item() # 预测正确的数目
    test_accs.append(100*correct/total)
print(np.array(test_accs).mean())

In [2]:
print(np.array(test_accs).mean())

58.578539823008846
